# Stage 1
****

In [36]:
# This script is the first stage of the project CSV2VCF-converter

# In this first stage, we pretend to write a code to be able to identify 
# different types of ID variants from a CSV input.

# Then, obtain data using this ID in a Request API

# Finally, generate an df with the data neccesary to write a VCF file.



In [1]:
import sys
import os
import requests
import chardet
import json
import pandas as pd
import numpy as np
import os.path

### To read input

In [136]:
# To read input

# This code is able to recognise the 4 type of ID variant mention above contained in a CSV file.

CSV_input = '/Users/manolodominguez/Desktop/git-repos/STP_mini_projects/Igor-Manuel/Igor-Manuel/different_input/NM_.csv'

## The row 4 and 5th contain errors

# input to pd data frame

# This is to avoid UnicodeDecodeError: 'utf-8'
with open(CSV_input, 'rb') as f:
    result = chardet.detect(f.read())

# This not work 100% of the times. To avoid UnicodeDecodeError: 'utf-8'
# We strongly recommend that the input be .csv

df = pd.read_csv(CSV_input, encoding=result['encoding'])

# We will need a extra copy to avoid repetitions
df_draft = df

print(' The row 4 and 5th contain errors')

df

 The row 4 and 5th contain errors


,Gene,GRCh38 coordinates,Transcript,Quality,Filter
0,IMPG2,3:101232831,NM_016247.3:c.3183C>G,34/64,PASS
1,BRCA1,17:43067616,NM_007294.3:c.5066T>C,49/131,PASS
2,FBN1,15:48600225,NM_000138.4:c.356G>A,46/99,PASS
3,MYH7,14:23426833,NM_000257.3:c.1988G>A,8/102,NO PASS
4,BORIS,9:133256042,rs56116432,1/200,NO PASS
5,BRCA1,77:43071077,rs1799966,14/85,PASS


### Here we read and found what we want from the CSV


In [137]:
##### Reading CSV file values and looking for variants IDs ######

# Our programme recognise hgvs_notation (e.g AGT:c.803T>C and ENST00000003084:c.1431_1433delTTC) 
# and also dbSNP ID (e.g rs17289390)


# Find Variant Transcript ':c.' in CSV
Transcript = df_draft[df_draft.apply(lambda x:x.str.contains(":c."))].dropna(how='all').dropna(axis=1, how='all')

# Now, we save the results found in a dict key=index and value=variand ID
if Transcript.empty == False:
    ind = Transcript.index.to_list()
    vals = list(Transcript.stack().values)
    row2Transcript = dict(zip(ind, vals))
    # We need to remove the row where rs has been found to avoid repetitions
    # In case in same row more than one kind of ID Variant is stored ('e.g :c.' and '(:p.)')   
    for index, Transcript  in row2Transcript.items(): 
        # This will be done in df_draft
        df_draft = df_draft.drop(index)
        
# Same but now with dbSNP variant ID
rs = df[df.apply(lambda x:x.str.contains("rs\d+"))].dropna(how='all').dropna(axis=1, how='all')

#rs = df_draft[df_draft.apply(lambda x:x.str.contains("rs"))].dropna(how='all').dropna(axis=1, how='all')

# Now, we save the results found in a dict key=index and value=variand ID
if rs.empty == False:
    ind = rs.index.to_list()
    vals = list(rs.stack().values)
    row2rs = dict(zip(ind, vals))
    # We need to remove the row where rs has been found to avoid repetitions
    # In case in same row more than one kind of ID Variant is stored ('e.g :c.' and '(:p.)')   
    for index, rs  in row2rs.items(): 
        # This will be done in df_draft
        df_draft = df_draft.drop(index)


        
# df_draft empty means every row contained a rs or :c. ID
print('Is the DataFrame empty? ', df_draft.empty)

Is the DataFrame empty?  True


### Let's see if last code worked

In [138]:
row2rs

{4: 'rs56116432', 5: 'rs1799966'}

In [139]:
row2Transcript

{0: 'NM_016247.3:c.3183C>G',
 1: 'NM_007294.3:c.5066T>C',
 2: 'NM_000138.4:c.356G>A',
 3: 'NM_000257.3:c.1988G>A'}

### Now, the request API

In [140]:
row2gene_symbol = dict()
row2CHROM = dict()
row2POS = dict()
row2ALT = dict()
row2REF = dict()


server = 'https://rest.ensembl.org/vep/human/hgvs/'

for row, transcript in row2Transcript.items():

    r = requests.get(server+transcript, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    decoded = r.json()[0]
    
# For row2gene_symbol
    row2gene_symbol[row]= decoded["transcript_consequences"][0]['gene_symbol']

# For row2CHROM
    row2CHROM[row] = decoded["seq_region_name"]
    
# For row2POS 
    row2POS[row] = decoded["start"]
    
# For row2ALT 
    row2ALT[row] = decoded["transcript_consequences"][0]['variant_allele']

# For row2REF
    row2REF[row] = decoded['allele_string'][0]
    
########################################################################    
# Here is where we need to put some exception if variant ID is unknown #
########################################################################

#### Let's see if the ditc work

In [141]:
row2gene_symbol

{0: 'IMPG2', 1: 'BRCA1', 2: 'FBN1', 3: 'MYH7'}

In [142]:
row2CHROM

{0: '3', 1: '17', 2: '15', 3: '14'}

In [143]:
row2POS

{0: 101232831, 1: 43067616, 2: 48600225, 3: 23426833}

In [144]:
row2ALT

{0: 'G', 1: 'C', 2: 'A', 3: 'A'}

In [145]:
row2REF

{0: 'C', 1: 'T', 2: 'G', 3: 'G'}

In [146]:
# Now, we need to do the same with row2rs

server = "https://rest.ensembl.org/vep/human/id/"

interrogation = '?'

for row, rs in row2rs.items():

    r = requests.get(server+rs+interrogation, headers={ "Content-Type" : "application/json"})
    if not r.ok:
        r.raise_for_status()
        sys.exit()
    decoded = r.json()[0]
    
    
# For row2gene_symbol
    row2gene_symbol[row]= decoded['transcript_consequences'][0]['gene_symbol']

# For row2CHROM
    row2CHROM[row] = decoded['seq_region_name']
    
# For row2POS 
    row2POS[row] = decoded['start']
    
# For row2ALT 
    row2ALT[row] = decoded['transcript_consequences'][0]['variant_allele']

# For row2REF
    row2REF[row] = decoded['allele_string'][0]
    


In [147]:
row2gene_symbol

{0: 'IMPG2', 1: 'BRCA1', 2: 'FBN1', 3: 'MYH7', 4: 'ABO', 5: 'BRCA1'}

In [148]:
row2CHROM

{0: '3', 1: '17', 2: '15', 3: '14', 4: '9', 5: '17'}

In [149]:
row2POS

{0: 101232831,
 1: 43067616,
 2: 48600225,
 3: 23426833,
 4: 133256042,
 5: 43071077}

In [150]:
row2ALT

{0: 'G', 1: 'C', 2: 'A', 3: 'A', 4: 'T', 5: 'A'}

In [151]:
row2REF

{0: 'C', 1: 'T', 2: 'G', 3: 'G', 4: 'C', 5: 'T'}

#### This looks ok

In [152]:
# To finish this stage, 
# Let's save our dicts in a df

column_names = ["Gene_API", 'CHROM','POS','ID','REF','ALT','QUAL','FILTER','INFO']
df_stage1 = pd.DataFrame(columns = column_names)

In [153]:
df_stage1 = pd.DataFrame.from_dict(dict(zip(column_names, [row2gene_symbol, row2CHROM, row2POS,df["Transcript"] ,row2ALT,row2REF, df["Quality"], df["Filter"]  ])))
df_stage1

,Gene_API,CHROM,POS,ID,REF,ALT,QUAL,FILTER
0,IMPG2,3,101232831,NM_016247.3:c.3183C>G,G,C,34/64,PASS
1,BRCA1,17,43067616,NM_007294.3:c.5066T>C,C,T,49/131,PASS
2,FBN1,15,48600225,NM_000138.4:c.356G>A,A,G,46/99,PASS
3,MYH7,14,23426833,NM_000257.3:c.1988G>A,A,G,8/102,NO PASS
4,ABO,9,133256042,rs56116432,T,C,1/200,NO PASS
5,BRCA1,17,43071077,rs1799966,A,T,14/85,PASS


# End of stage 1

# Stage 2
****

In [154]:
# in Stage 2 we compare Gene and location between input and API request.
# And we create a txt file to inform error found.


In [272]:
# First, let's prepare where our information file is going to be create.

# Introce in path the directory where we wish to create the new CVF
save_path = '/Users/manolodominguez/Desktop/git-repos/STP_mini_projects/Igor-Manuel/Igor-Manuel/Output'

# Introduce in name, the name of your new file
name_of_file= "inform_error"

# This will  create directory + file name
completeName = os.path.join(save_path, name_of_file+".txt")         





In [156]:
# To compare input with API result
# We need to create a new column from CHROM + ':' + POS

df_stage1 = df_stage1.applymap(str)
df_stage1['Location'] = df_stage1['CHROM'].str.cat(df_stage1['POS'], sep=':')
df_stage1

,Gene_API,CHROM,POS,ID,REF,ALT,QUAL,FILTER,Location
0,IMPG2,3,101232831,NM_016247.3:c.3183C>G,G,C,34/64,PASS,3:101232831
1,BRCA1,17,43067616,NM_007294.3:c.5066T>C,C,T,49/131,PASS,17:43067616
2,FBN1,15,48600225,NM_000138.4:c.356G>A,A,G,46/99,PASS,15:48600225
3,MYH7,14,23426833,NM_000257.3:c.1988G>A,A,G,8/102,NO PASS,14:23426833
4,ABO,9,133256042,rs56116432,T,C,1/200,NO PASS,9:133256042
5,BRCA1,17,43071077,rs1799966,A,T,14/85,PASS,17:43071077


In [157]:
# Now, we can compare Gene and location of the input againt Gene and location
# from the API

# Let's create a new df for comparison purpose

df_comparation =  pd.concat([df.iloc[:, 0:3], df_stage1[['Gene_API','Location']]], axis=1)
#df_comparation = df_comparation.set_index("Transcript")

In [158]:
df_comparation

,Gene,GRCh38 coordinates,Transcript,Gene_API,Location
0,IMPG2,3:101232831,NM_016247.3:c.3183C>G,IMPG2,3:101232831
1,BRCA1,17:43067616,NM_007294.3:c.5066T>C,BRCA1,17:43067616
2,FBN1,15:48600225,NM_000138.4:c.356G>A,FBN1,15:48600225
3,MYH7,14:23426833,NM_000257.3:c.1988G>A,MYH7,14:23426833
4,BORIS,9:133256042,rs56116432,ABO,9:133256042
5,BRCA1,77:43071077,rs1799966,BRCA1,17:43071077


In [159]:
# New two columns as result of the comparation
df_comparation['result1'] = np.where(df_comparation.iloc[:, 0] == df_comparation.iloc[:, 3], 'OK', 'ERROR')

In [160]:
df_comparation['result2'] = np.where(df_comparation.iloc[:, 1] == df_comparation.iloc[:, 4], 'OK', 'ERROR')

In [161]:
df_comparation

,Gene,GRCh38 coordinates,Transcript,Gene_API,Location,result1,result2
0,IMPG2,3:101232831,NM_016247.3:c.3183C>G,IMPG2,3:101232831,OK,OK
1,BRCA1,17:43067616,NM_007294.3:c.5066T>C,BRCA1,17:43067616,OK,OK
2,FBN1,15:48600225,NM_000138.4:c.356G>A,FBN1,15:48600225,OK,OK
3,MYH7,14:23426833,NM_000257.3:c.1988G>A,MYH7,14:23426833,OK,OK
4,BORIS,9:133256042,rs56116432,ABO,9:133256042,ERROR,OK
5,BRCA1,77:43071077,rs1799966,BRCA1,17:43071077,OK,ERROR


In [162]:
# We save here the rows where errors have been in gene
gene_error = df_comparation[df_comparation['result1'].str.match('ERROR')]
gene_error = gene_error.drop(gene_error.columns[[1,4,5,6]], axis=1)

In [274]:
gene_error

,Gene,Transcript,Gene_API
4,BORIS,rs56116432,ABO


In [164]:
# we save this information in a dict with the variant where the error has been found as key
# Plus in a list the gene simbol of the input and the gene simbol of the API as value.
gene2gene_API = gene_error.set_index('Transcript').T.to_dict('list')
gene2gene_API

{'rs56116432': ['BORIS', 'ABO']}

In [165]:
# Same operation for location

In [181]:
location_error = df_comparation[df_comparation['result2'].str.match('ERROR')]
location_error = location_error.drop(location_error.columns[[0,3,5,6]], axis=1)
location_error

,GRCh38 coordinates,Transcript,Location
5,77:43071077,rs1799966,17:43071077


In [167]:
variant2locations = location_error.set_index('Transcript').T.to_dict('list')
variant2locations

{'rs1799966': ['77:43071077', '17:43071077']}

In [297]:
# Now with dicts we can write the inform_error.txt
# If both dicts are empty
# then create the txt saying there is not error found
# If one or both dict contain data
# then says the errors found


# If there is not any mismatch found, one document will be created saying 
No_error_found = 'Congratulation, no errors have been found in your CSV with regard to gene symbol and location\n'


# If there are mismathes found, one document will be created saying
Errors_found = 'If you are reading this document is because some error(s) or mismatch(s) has been found between the Gene Symbol and/or the location of your variants\nWhen these has been compared with API Emsembl.\nWe show you the differences found in this document.\n\n'
# Plus the mismatch


# Number of locations mismatches 
how_many_location_error = 'Number of location mismachts found:', str(len(location_error))
how_many_location_error = ' '.join(how_many_location_error)

# Now, show these mismatchs in this order

header_location = 'Your location     variant    API location\n'

# Now the same with gene_symbol

# Number of gene_symbol mismatchs
how_many_GS_error = 'Number of Gene Symbol mismachts found:', str(len(gene_error))
how_many_GS_error = ' '.join(how_many_GS_error)

# Now, show these mismatchs in this order

header_gene_symbol = 'Your Gene_symbol,     Variant,       API Gene_symbol\n'
separator = '\n'

location_title = "############# LOCATION'S MISMACHTS ############## \n\n"
gene_title = "\n\n############# GENE_SYMBOL'S MISMACHTS #############\n\n"

# Here we introduce a condition, if dicts are empty
# write No_error_found
# if dicts are. not empty, write all this information

if bool(variant2locations) == False:
    if bool(gene2gene_API)==False:
        with open(completeName,'w') as out:
            out.writelines([No_error_found])
else:
    # Write location
    
    with open(completeName,'w') as out:
            out.writelines([Errors_found, location_title, how_many_location_error, separator, header_location])   
    location_error.to_csv(completeName,
          header=None, index=None, sep='\t', mode='a')

    
    # Now, with Gene_symbol    
    
    with open(completeName,'a') as out:
        out.writelines([gene_title, how_many_GS_error, separator,header_gene_symbol])
    gene_error.to_csv(completeName,
          header=None, index=None, sep='\t', mode='a')

In [258]:
how_many_gene_error = 'Number of mismachts found', str(len(gene2gene_API))


In [265]:
type(how_many_gene_error)

str

In [264]:
how_many_gene_error = ' '.join(how_many_gene_error)